In [1]:
import uproot
import subprocess
import awkward as ak
import numpy as np
import hist
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

In [2]:
hlt_paths_used = ["HLT_IsoTkMu24", 
"HLT_IsoMu24", 
"HLT_IsoMu27", "HLT_IsoMu19_eta2p1_LooseIsoPFTau20_SingleL1", 
"HLT_IsoMu20_eta2p1_LooseChargedIsoPFTau27_eta2p1_CrossL1", 
"HLT_IsoMu20_eta2p1_LooseChargedIsoPFTauHPS27_eta2p1_CrossL1", 
"HLT_Ele25_eta2p1_WPTight_Gsf", 
"HLT_Ele32_WPTight_Gsf",
"HLT_Ele32_WPTight_Gsf_L1DoubleEG", 
"HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTau30_eta2p1_CrossL1", 
"HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTauHPS30_eta2p1_CrossL1", 
"HLT_DoubleMediumIsoPFTau35_Trk1_eta2p1_Reg", 
"HLT_DoubleMediumCombinedIsoPFTau35_Trk1_eta2p1_Reg", 
"HLT_DoubleMediumChargedIsoPFTau40_Trk1_TightID_eta2p1_Reg", 
"HLT_DoubleTightChargedIsoPFTau40_Trk1_eta2p1_Reg",
"HLT_DoubleMediumChargedIsoPFTauHPS35_Trk1_eta2p1_Reg", 
"HLT_DoubleMediumDeepTauPFTauHPS35_L2NN_eta2p1"]
needed_branches = hlt_paths_used + ["GenPart_pdgId", "GenPart_genPartIdxMother", "GenPart_eta", "GenPart_pt", "nGenVisTau", "GenVisTau_pt", "GenVisTau_eta"]
zz_signal_das = "/ZZTo2Q2L_mllmin4p0_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM"
zbb_htt_signal_das = "/ZHToTauTau_M125_CP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NANOAODSIM"
ztt_hbb_signal_das = "/ZH_HToBB_ZToLL_M-125_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM"

In [3]:
def getFilesForDas(dataset_das):
    res = subprocess.run(["dasgoclient", "-query", "file dataset=" + dataset_das], capture_output=True, text=True, check=True)
    return ["root://xrootd-cms.infn.it//" + path for path in res.stdout.split("\n")]
def concatFiles(paths):
    ars = []
    for path in paths:
        try:
            ars.append(uproot.open(path, timeout=15).arrays(filter_name=needed_branches, how="zip"))
        except OSError as e:
            print(e)
    return ak.concatenate(ars)
def filter_ZToBB(ar):
    return ak.any((abs(ar.GenPart.pdgId) == 5) & (abs(ar.GenPart[ar.GenPart.genPartIdxMother].pdgId) == 23), axis=-1) # b decaying from Z
def filter_ZToTauTau(ar):
    return ak.any((abs(ar.GenPart.pdgId) == 15) & (abs(ar.GenPart[ar.GenPart.genPartIdxMother].pdgId) == 23), axis=-1) # tau decaying from Z
def filter_ZToBB_ZToTauTau(ar):
    # tau decaying from Z & b decaying from Z
    return ak.any((abs(ar.GenPart.pdgId) == 15) & (abs(ar.GenPart[ar.GenPart.genPartIdxMother].pdgId) == 23), axis=-1) & ak.any((abs(ar.GenPart.pdgId) == 5) & (abs(ar.GenPart[ar.GenPart.genPartIdxMother].pdgId) == 23), axis=-1) 
def genPairType_branch(ar):
    """ select etau, mutau, tautau at genlevel"""
    tautau = ar.nGenVisTau == 2

    mutau_genMuon_bool = (abs(ar.GenPart.pdgId) == 13) & (abs(ar.GenPart[ar.GenPart.genPartIdxMother].pdgId) == 15)
    mutau_genMuon_idx = ak.firsts(ak.local_index(ar.GenPart)[mutau_genMuon_bool]) # index of gen muon in GenPart collection
    mutau = (ar.nGenVisTau == 1) & ak.any(mutau_genMuon_bool, axis=-1)
    
    etau_genMuon_bool = (abs(ar.GenPart.pdgId) == 11) & (abs(ar.GenPart[ar.GenPart.genPartIdxMother].pdgId) == 15)
    etau_genMuon_idx = ak.firsts(ak.local_index(ar.GenPart)[etau_genMuon_bool]) # index of gen muon in GenPart collection
    etau = (ar.nGenVisTau == 1) & ak.any(etau_genMuon_bool, axis=-1)
    
    genPairType =  ak.where(mutau, 0, ak.where(etau, 1, ak.where(tautau, 2, -1)))
    genLeptonIdx = ak.where(mutau, mutau_genMuon_idx, etau_genMuon_idx) # for tautau will just put None
    return ak.with_field(ak.with_field(ar, genPairType, where="genPairType"), genLeptonIdx, "genLeptonIdx")

In [5]:
zbb_htt_signal = concatFiles([path + ":Events" for path in getFilesForDas(zbb_htt_signal_das)[:2]])
zbb_htt_signal = genPairType_branch(zbb_htt_signal[filter_ZToBB(zbb_htt_signal)])
zbb_htt_signal

File stat request failed: [ERROR] Server responded with an error: [3010] Unable to locate /store/mc/RunIISummer20UL18NanoAODv9/ZHToTauTau_M125_CP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/50000/67CB0663-4BFE-DA41-BA3C-6AC80E8F60D9.root; permission denied

File stat request failed: [ERROR] Server responded with an error: [3010] Unable to locate /store/mc/RunIISummer20UL18NanoAODv9/ZHToTauTau_M125_CP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/40000/9714A31C-6A35-DD44-96D5-89312CCB1796.root; permission denied



ValueError: need at least one array to concatenate

In [5]:
ztt_hbb_signal = concatFiles([path + ":Events" for path in getFilesForDas(ztt_hbb_signal_das)[:10]])
ztt_hbb_signal = genPairType_branch(ztt_hbb_signal[filter_ZToTauTau(ztt_hbb_signal)])

KeyboardInterrupt: 

In [ ]:
zz_signal = concatFiles([path + ":Events" for path in getFilesForDas(zz_signal_das)[:10]])
zz_signal = genPairType_branch(zz_signal[filter_ZToBB_ZToTauTau(zz_signal)])

Bytes failed to read from open file: [ERROR] Operation expired
Bytes failed to read from open file: [ERROR] Operation expired
Bytes failed to read from open file: [ERROR] Operation expired
Bytes failed to read from open file: [ERROR] Operation expired
Bytes failed to read from open file: [ERROR] Operation expired
Bytes failed to read from open file: [ERROR] Operation expired
File did not open properly: [ERROR] Operation expired
Bytes failed to read from open file: [ERROR] Operation expired
Bytes failed to read from open file: [ERROR] Operation expired


In [6]:
def eff(branch_num, filter_denom):
    return ak.count_nonzero(branch_num) / ak.count_nonzero(filter_denom)

def getAvailHLT(ar):
    return [f for f in ar.fields if f.startswith("HLT_")]
def printEffs(ar):
    paths = getAvailHLT(ar)
    for path in paths:
        print(f"{path} : {ak.count_nonzero(ar[path])/len(ar):.1%}")
def filterEvents_singleLepton(ar, genPairType, pt_cut, eta_cut):
    """ filter events for lepton triggers, putslk (<- Marina Maroni here) a cut on gen muon/electron"""
    filter_genPairType = ar.genPairType == genPairType
    gen_leptons = ak.firsts(ar.GenPart[ak.singletons(ar.genLeptonIdx)])
    # fill_none for cases where no gen lepton (ie tautau)
    return ak.fill_none(filter_genPairType & (gen_leptons.pt >= pt_cut) & (gen_leptons.eta <= eta_cut), False)
def filterEvents_cross(ar, genPairType, pt_cut_lepton, eta_cut_lepton, pt_cut_tau, eta_cut_tau):
    filter_genPairType = ar.genPairType == genPairType
    gen_leptons = ak.firsts(ar.GenPart[ak.singletons(ar.genLeptonIdx)])
    gen_taus = ak.firsts(ar.GenVisTau)
    # fill_none for cases where no gen lepton (ie tautau)
    return ak.fill_none(filter_genPairType & (gen_leptons.pt >= pt_cut_lepton) & (gen_leptons.eta <= eta_cut_lepton) & (gen_taus.pt >= pt_cut_tau) & (gen_taus.eta <= eta_cut_tau), False)
def filterEvents_diTau(ar, pt_cut, eta_cut):
    filter_genPairType = ar.genPairType == 2
    genVisTau_pt_padded = ak.pad_none(ar.GenVisTau.pt, 2)
    genVisTau_eta_padded = ak.pad_none(ar.GenVisTau.eta, 2)
    return filter_genPairType & (genVisTau_pt_padded[:, 0] >= pt_cut) & (genVisTau_pt_padded[:, 1] >= pt_cut) & (genVisTau_eta_padded[:, 0] <= eta_cut) & (genVisTau_eta_padded[:, 1] <= eta_cut)

In [7]:
def printEffs_mutau(signal):
    base_filter_mutau = filterEvents_singleLepton(signal, 0, 20, 2.4)
    print(f"Baseline eff {eff(base_filter_mutau, signal.genPairType == 0):.1%}")
    print(f"Single-mu eff {eff(signal[base_filter_mutau].HLT_IsoMu24, base_filter_mutau):.1%}")
    print(f"Cross-mu eff {eff(signal[base_filter_mutau].HLT_IsoMu20_eta2p1_LooseChargedIsoPFTauHPS27_eta2p1_CrossL1, base_filter_mutau):.1%}")
    print(f"Full OR eff {eff(signal[base_filter_mutau].HLT_IsoMu24|signal[base_filter_mutau].HLT_IsoMu20_eta2p1_LooseChargedIsoPFTauHPS27_eta2p1_CrossL1, base_filter_mutau):.1%}")
    # print(f"Cross-mu eff {eff(zbb_htt_signal[base_filter_mutau].HLT_IsoMu20_eta2p1_LooseChargedIsoPFTauHPS27_eta2p1_CrossL1, base_filter_mutau):.1%}")
def printEffs_etau(signal):
    base_filter_etau = filterEvents_singleLepton(signal, 1, 24, 2.5)
    print(f"Baseline eff {eff(base_filter_etau, signal.genPairType == 1):.1%}")
    print(f"Single-e eff {eff(signal[base_filter_etau].HLT_Ele32_WPTight_Gsf, base_filter_etau):.1%}")
    print(f"Cross-e eff {eff(signal[base_filter_etau].HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTauHPS30_eta2p1_CrossL1, base_filter_etau):.1%}")
    print(f"Full OR eff {eff(signal[base_filter_etau].HLT_Ele32_WPTight_Gsf|signal[base_filter_etau].HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTauHPS30_eta2p1_CrossL1, base_filter_etau):.1%}")
def printEffs_tautau(signal):
    base_filter_tautau = filterEvents_diTau(signal, 35, 2.1)
    print(f"Baseline eff {eff(base_filter_tautau, signal.genPairType == 2):.1%}")
    print(f"Ditau eff {eff(signal[base_filter_tautau].HLT_DoubleMediumChargedIsoPFTauHPS35_Trk1_eta2p1_Reg, base_filter_tautau):.1%}")


In [11]:
printEffs_mutau(zbb_htt_signal)
printEffs_etau(zbb_htt_signal)
printEffs_tautau(zbb_htt_signal)

Baseline eff 39.5%
Single-mu eff 65.3%
Cross-mu eff 29.9%
Full OR eff 69.9%
Baseline eff 33.6%
Single-e eff 44.6%
Cross-e eff 18.6%
Full OR eff 49.7%
Baseline eff 22.4%
Ditau eff 25.8%


In [12]:
printEffs_mutau(ztt_hbb_signal)
printEffs_etau(ztt_hbb_signal)
printEffs_tautau(ztt_hbb_signal)

Baseline eff 32.2%
Single-mu eff 63.4%
Cross-mu eff 27.2%
Full OR eff 68.0%
Baseline eff 26.2%
Single-e eff 42.4%
Cross-e eff 15.9%
Full OR eff 46.8%
Baseline eff 14.8%
Ditau eff 27.9%


In [13]:
printEffs_mutau(zz_signal)
printEffs_etau(zz_signal)
printEffs_tautau(zz_signal)

Baseline eff 29.2%
Single-mu eff 61.1%
Cross-mu eff 20.3%
Full OR eff 64.3%
Baseline eff 22.3%
Single-e eff 34.8%
Cross-e eff 11.2%
Full OR eff 39.7%
Baseline eff 9.4%
Ditau eff 21.0%


In [17]:
ak.count_nonzero(zz_signal[filterEvents_singleLepton(zz_signal, 1, 24, 2.5)].HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTauHPS30_eta2p1_CrossL1)

72

In [92]:
# looking at intersection of phase space of single and cross triggers
base_filter_mutau = filterEvents_singleLepton(zbb_htt_signal, 0, 20, 2.4)
# f_overlap = f_singleMu & f_crossMu
# f_singleMu_overlap = filterEvents_singleLepton(zbb_htt_signal, 0, 26, 2.1)
# f_crossMu_overlap = filterEvents_cross(zbb_htt_signal, 0, 26, 2.1, 32, 2.1)
# (singleMu|cross, singleMu, singleMu&noCross, noSingleMu&cross)
ak.count_nonzero(zbb_htt_signal[f_overlap]["HLT_IsoMu24"] | zbb_htt_signal[f_overlap]["HLT_IsoMu20_eta2p1_LooseChargedIsoPFTauHPS27_eta2p1_CrossL1"])/ak.count_nonzero(f_overlap), ak.count_nonzero(zbb_htt_signal[f_overlap]["HLT_IsoMu24"])/ak.count_nonzero(f_overlap), ak.count_nonzero(zbb_htt_signal[f_overlap]["HLT_IsoMu24"] & (~(zbb_htt_signal[f_overlap]["HLT_IsoMu20_eta2p1_LooseChargedIsoPFTauHPS27_eta2p1_CrossL1"])))/ak.count_nonzero(f_overlap), ak.count_nonzero((~(zbb_htt_signal[f_overlap]["HLT_IsoMu24"])) & zbb_htt_signal[f_overlap]["HLT_IsoMu20_eta2p1_LooseChargedIsoPFTauHPS27_eta2p1_CrossL1"])/ak.count_nonzero(f_overlap)

(0.8021372752094906,
 0.7985123811317202,
 0.26179267488937014,
 0.0036248940777704546)

In [18]:
ak.count_nonzero(zbb_htt_signal[base_filter_mutau]["HLT_IsoMu20_eta2p1_LooseChargedIsoPFTauHPS27_eta2p1_CrossL1"] &(~zbb_htt_signal[base_filter_mutau]["HLT_IsoMu24"])), ak.count_nonzero(base_filter_mutau)

(7, 7756)

In [14]:
base_filter_mutau = filterEvents_singleLepton(zbb_htt_signal, 0, 26, 2.4)
eff(zbb_htt_signal[base_filter_mutau]["HLT_IsoMu20_eta2p1_LooseChargedIsoPFTauHPS27_eta2p1_CrossL1"] &(~zbb_htt_signal[base_filter_mutau]["HLT_IsoMu24"]), base_filter_mutau)

0.0009025270758122744

In [16]:
base_filter_etau = filterEvents_singleLepton(zbb_htt_signal, 1, 33, 2.5)
eff((~(zbb_htt_signal[base_filter_etau].HLT_Ele32_WPTight_Gsf)) &zbb_htt_signal[base_filter_etau].HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTauHPS30_eta2p1_CrossL1, base_filter_etau)

0.0076676176890156916

In [26]:
printEffs(zbb_htt_signal)

HLT_Ele32_WPTight_Gsf_L1DoubleEG : 8.1%
HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTauHPS30_eta2p1_CrossL1 : 3.4%
HLT_IsoMu20_eta2p1_LooseChargedIsoPFTauHPS27_eta2p1_CrossL1 : 5.0%
HLT_IsoMu24 : 11.9%
HLT_IsoMu27 : 10.9%
HLT_DoubleMediumChargedIsoPFTauHPS35_Trk1_eta2p1_Reg : 5.1%
